In [8]:
# USE_IBM_RUNTIME = True  # False = lokální výpočet

# from qiskit_algorithms import VQE
# from qiskit_algorithms.optimizers import SLSQP
# from qiskit_aer import AerSimulator
# from qiskit import transpile
# from qiskit.circuit.library import TwoLocal
# from qiskit.quantum_info import SparsePauliOp

# if USE_IBM_RUNTIME:
#     from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
#     service = QiskitRuntimeService()
#     estimator = EstimatorV2(service=service, backend="ibmq_qasm_simulator")
# else:
#     from qiskit.primitives import Estimator
#     estimator = Estimator()

# # -----------------------------
# # Definice H2 Hamiltonianu přes SparsePauliOp
# h2_paulis = [
#     ("II", -1.052373245772859),
#     ("ZI",  0.3979374248431808),
#     ("IZ", -0.3979374248431808),
#     ("ZZ", -0.01128010425623538),
#     ("XX",  0.18093119978423156),
#     ("YY",  0.18093119978423156)
# ]
# h2_hamiltonian = SparsePauliOp.from_list(h2_paulis)

# # -----------------------------
# # Ansatz (jednoduchý obvod)
# ansatz = TwoLocal(num_qubits=2, rotation_blocks='ry', entanglement_blocks='cz', reps=1)

# # -----------------------------
# # VQE
# vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=SLSQP())
# vqe.initial_point = [0.0] * ansatz.num_parameters
# result = vqe.compute_minimum_eigenvalue(h2_hamiltonian)

# # -----------------------------
# # Výsledky
# print(f"Electronic energy: {result.eigenvalue.real:.6f} Ha")

# # -----------------------------
# # Circuit diagram
# ansatz.draw(output='mpl')


In [1]:
USE_IBM_RUNTIME = True  # změň na False, pokud chceš lokální výpočet

from qiskit_nature.units import DistanceUnit
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

if USE_IBM_RUNTIME:
    from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
    service = QiskitRuntimeService()
    estimator = EstimatorV2(service=service, backend="ibmq_qasm_simulator")
else:
    from qiskit.primitives import Estimator
    estimator = Estimator()

# Molekulární systém
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()

# Fermionický Hamiltonian
ferm_hamiltonian = es_problem.hamiltonian.second_q_op()

# Mapování na qubity
mapper = JordanWignerMapper()
qubit_op = mapper.map(ferm_hamiltonian)

# Ansatz
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

# VQE
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters
result = vqe.compute_minimum_eigenvalue(qubit_op)

# Výsledky
E_elec = result.eigenvalue.real
E_total = E_elec + es_problem.nuclear_repulsion_energy

print(f"Electronic energy: {E_elec:.6f} Ha")
print(f"Nuclear repulsion: {es_problem.nuclear_repulsion_energy:.6f} Ha")
print(f"Total energy:     {E_total:.6f} Ha")

# Nakreslit ansatz
ansatz.draw(output="mpl")


ImportError: cannot import name 'CONTROL_FLOW_OP_NAMES' from 'qiskit.circuit' (/Users/ema/Documents/Python/quantum/Quantum-Examples/.venv/lib/python3.11/site-packages/qiskit/circuit/__init__.py)

In [2]:
USE_IBM_RUNTIME = True  # změň na False pro lokální výpočet

from qiskit_nature.units import DistanceUnit
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

if USE_IBM_RUNTIME:
    from qiskit_ibm_runtime import QiskitRuntimeService
    from qiskit.primitives import Estimator
    service = QiskitRuntimeService()
    estimator = Estimator(session=service, options={"backend_name": "ibmq_qasm_simulator"})
else:
    from qiskit.primitives import Estimator
    estimator = Estimator()

# Molekulární systém
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
es_problem = driver.run()

# Fermionický Hamiltonian
ferm_hamiltonian = es_problem.hamiltonian.second_q_op()

# Mapování na qubity
mapper = JordanWignerMapper()
qubit_op = mapper.map(ferm_hamiltonian)

# Ansatz
ansatz = UCCSD(
    num_spatial_orbitals=es_problem.num_spatial_orbitals,
    num_particles=es_problem.num_particles,
    mapper=mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals=es_problem.num_spatial_orbitals,
        num_particles=es_problem.num_particles,
        mapper=mapper,
    ),
)

# VQE
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters
result = vqe.compute_minimum_eigenvalue(qubit_op)

# Výsledky
E_elec = result.eigenvalue.real
E_total = E_elec + es_problem.nuclear_repulsion_energy

print(f"Electronic energy: {E_elec:.6f} Ha")
print(f"Nuclear repulsion: {es_problem.nuclear_repulsion_energy:.6f} Ha")
print(f"Total energy:     {E_total:.6f} Ha")

# Nakreslit ansatz
ansatz.draw(output="mpl")


ImportError: cannot import name 'CONTROL_FLOW_OP_NAMES' from 'qiskit.circuit' (/Users/ema/Documents/Python/quantum/Quantum-Examples/.venv/lib/python3.11/site-packages/qiskit/circuit/__init__.py)

ImportError: cannot import name 'Aer' from 'qiskit' (/Users/ema/Documents/Python/quantum/Quantum-Examples/qiskit_env/lib/python3.11/site-packages/qiskit/__init__.py)

In [1]:
# Základní Qiskit
from qiskit import QuantumCircuit, Aer, execute

# Algoritmy
from qiskit_algorithms import VQE, QAOA
from qiskit_algorithms.optimizers import COBYLA

# Nature modul (chemie, fyzika)
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

# IBM Runtime pro práci s backendem
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler


ImportError: cannot import name 'Aer' from 'qiskit' (/Users/ema/Documents/Python/quantum/Quantum-Examples/qiskit_env/lib/python3.11/site-packages/qiskit/__init__.py)